<a href="https://colab.research.google.com/github/noafrimerman/BCI/blob/main/bci2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# Installing MNE (eeg package)
!pip install mne

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 81.1 MB/s eta 0:00:00


In [3]:
"""
Create movement annotations for EEG BCI dataset.

This function detects movement-direction events (Right, Left, BothHands, Neutral)
based on either the target trajectory or the cursor trajectory.
The user can choose which signal to follow using follow="target" or follow="cursor".

Additionally, all detected events can be forced to have a fixed duration,
defined by the parameter fixed_event_length (in seconds).

Parameters
----------
raw : mne.io.Raw
    EEG data object to annotate.
target_x, target_y : array-like
    Target positions sampled at low resolution (usually 25 Hz).
cursor_x, cursor_y : array-like
    Cursor positions sampled at the same resolution.
follow : str
    Determines which position data to follow: "target" or "cursor".
fixed_event_length : float or None
    If given, all events will be truncated or extended to this duration.
eps_x, eps_y : float
    Thresholds for detecting meaningful movement in X/Y directions.
sfreq_positions : int
    Sampling rate of the position samples (default: 25 Hz).

Returns
-------
raw : mne.io.Raw
    Updated Raw object with annotations inserted.
events : list of dict
    List of detected events.
"""

import numpy as np
import mne


def moving_average(x, k=5):
    k = max(1, int(k))
    return np.convolve(x, np.ones(k) / k, mode='same')


def label_direction(dx, dy, eps_x, eps_y):
    if abs(dy) > eps_y:
        return "BothHands"
    if dx > eps_x:
        return "Right"
    if dx < -eps_x:
        return "Left"
    return "Neutral"


def add_motion_annotations(
    raw,
    target_x,
    target_y,
    cursor_x=None,
    cursor_y=None,
    follow="target",
    fixed_event_length=None,
    eps_x=0.01,
    eps_y=0.015,
    sfreq_positions=25
):
    if follow == "target":
        x = np.array(target_x)
        y = np.array(target_y)
    elif follow == "cursor":
        if cursor_x is None or cursor_y is None:
            raise ValueError("cursor_x and cursor_y must be provided when follow='cursor'.")
        x = np.array(cursor_x)
        y = np.array(cursor_y)
    else:
        raise ValueError("follow must be either 'target' or 'cursor'.")

    dx = np.diff(x)
    dy = np.diff(y)

    dx_s = moving_average(dx, 3)
    dy_s = moving_average(dy, 3)

    states = np.array([
        label_direction(dx_s[i], dy_s[i], eps_x, eps_y)
        for i in range(len(dx_s))
    ])

    events = []
    onset_list = []
    duration_list = []
    desc_list = []

    last_state = states[0]
    last_idx = 0

    for i in range(1, len(states)):
        if states[i] != last_state:
            if last_state != "Neutral":

                onset_sec = last_idx / sfreq_positions
                duration_sec = (i - last_idx) / sfreq_positions

                if fixed_event_length is not None:
                    duration_sec = fixed_event_length

                events.append({
                    "onset_s": onset_sec,
                    "duration_s": duration_sec,
                    "label": last_state
                })

                onset_list.append(onset_sec)
                duration_list.append(duration_sec)
                desc_list.append(last_state)

            last_state = states[i]
            last_idx = i

    if len(events) > 0:
        annotations = mne.Annotations(
            onset=onset_list,
            duration=duration_list,
            description=desc_list
        )
        raw.set_annotations(annotations)

    return raw, events


In [4]:
import os
import h5py
import numpy as np
import mne

def mat_to_raw(mat_path):
    with h5py.File(mat_path, "r") as f:
        eeg = f["eeg"]
        data = np.array(eeg["data"])
        data = data.astype(float)

        ch_names = [f"Ch{i+1}" for i in range(data.shape[0])]
        sfreq = int(np.array(eeg["fs"]))

        info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types="eeg")
        raw = mne.io.RawArray(data, info)

        target_x = eeg["targetpos"]["x"][:].flatten()
        target_y = eeg["targetpos"]["y"][:].flatten()
        cursor_x = eeg["cursorpos"]["x"][:].flatten()
        cursor_y = eeg["cursorpos"]["y"][:].flatten()

    return raw, target_x, target_y, cursor_x, cursor_y


def build_data_dict_from_folder(folder, eps_x, eps_y, fixed_event_length=None, follow="target"):
    data_dict = {}

    for filename in os.listdir(folder):
        if not filename.endswith(".mat"):
            continue

        mat_path = os.path.join(folder, filename)

        subject = filename.split("_")[0]
        session = filename.split("_")[1]
        session_num = int(session.replace("Se", ""))

        raw, tx, ty, cx, cy = mat_to_raw(mat_path)

        raw, events = add_motion_annotations(
            raw,
            target_x=tx,
            target_y=ty,
            cursor_x=cx,
            cursor_y=cy,
            follow=follow,
            eps_x=eps_x,
            eps_y=eps_y,
            fixed_event_length=fixed_event_length
        )

        if subject not in data_dict:
            data_dict[subject] = {}

        data_dict[subject][session_num] = (raw, events)

    return data_dict


In [ ]:
"""
Example usage:
Running the full pipeline: MAT → Raw → Movement detection → Annotations → data_dict
"""

# Path to your folder with MAT files
folder_path = "/content/drive/MyDrive/BCI/mat_files/SO1"

# Parameters chosen earlier from eps-analysis
eps_x = 0.0138
eps_y = 0.0133

# Event duration in seconds (set to None if you want the natural durations)
fixed_event_length = 1.0   # for example, 1-second windows
# fixed_event_length = None  # use natural durations instead

# Choose which trajectory to follow ("target" or "cursor")
follow_type = "target"

# Run the pipeline
print("Building data_dict ...")
data_dict = build_data_dict_from_folder(
    folder=folder_path,
    eps_x=eps_x,
    eps_y=eps_y,
    fixed_event_length=fixed_event_length,
    follow=follow_type
)

print("Done.")
print("Subjects:", data_dict.keys())

# Example: print info about subject S01, session 1
if "S01" in data_dict and 1 in data_dict["S01"]:
    raw_example, events_example = data_dict["S01"][1]
    print("\nExample: S01 Session 1")
    print("Number of detected events:", len(events_example))
    print("First events:", events_example[:5])

    # Show raw + annotations
    raw_example.plot()


Building data_dict ...


/tmp/ipython-input-749803483.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sfreq = int(np.array(eeg["fs"]))


Creating RawArray with float64 data, n_channels=317280, n_times=62
    Range : 0 ... 61 =      0.000 ...     0.061 secs
Ready.


/tmp/ipython-input-3101334479.py:126: RuntimeWarning: Omitted 146 annotation(s) that were outside data range.
  raw.set_annotations(annotations)
/tmp/ipython-input-749803483.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sfreq = int(np.array(eeg["fs"]))


Creating RawArray with float64 data, n_channels=317240, n_times=62
    Range : 0 ... 61 =      0.000 ...     0.061 secs
Ready.


...

/tmp/ipython-input-3101334479.py:126: RuntimeWarning: Omitted 179 annotation(s) that were outside data range.
  raw.set_annotations(annotations)


Creating RawArray with float64 data, n_channels=317280, n_times=62
    Range : 0 ... 61 =      0.000 ...     0.061 secs
Ready.


/tmp/ipython-input-3101334479.py:126: RuntimeWarning: Omitted 164 annotation(s) that were outside data range.
  raw.set_annotations(annotations)


Creating RawArray with float64 data, n_channels=317280, n_times=62
    Range : 0 ... 61 =      0.000 ...     0.061 secs
Ready.
Done.
Subjects: dict_keys(['S01'])

Example: S01 Session 1
Number of detected events: 162
First events: [{'onset_s': 2.16, 'duration_s': 1.0, 'label': np.str_('BothHands')}, {'onset_s': 16.8, 'duration_s': 1.0, 'label': np.str_('Right')}, {'onset_s': 17.32, 'duration_s': 1.0, 'label': np.str_('Right')}, {'onset_s': 19.12, 'duration_s': 1.0, 'label': np.str_('BothHands')}, {'onset_s': 19.28, 'duration_s': 1.0, 'label': np.str_('BothHands')}]


/tmp/ipython-input-3101334479.py:126: RuntimeWarning: Omitted 162 annotation(s) that were outside data range.
  raw.set_annotations(annotations)
